# USA Housing Listings
Source: [Kaggle](https://www.kaggle.com/austinreese/usa-housing-listings)

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from joblib import dump
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

In [ ]:
filename = "kaggle/housing.csv"

In [ ]:
#df = pd.read_csv(filename, skiprows=10, nrows=10)
df = pd.read_csv(filename)

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.count()

In [ ]:
def doit(X, y, name):
    x_train, x_test, y_train, y_test = train_test_split(X, y, random_state=4)
    #model = LinearRegression()
    model = DecisionTreeRegressor(random_state=1)
    model.fit(x_train, y_train)
    print(name)
    print(f"Number of rows: {X.size}")
    print(f"train coefficient of determination: {model.score(x_train, y_train)}")
    print(f"test  coefficient of determination: {model.score(x_test, y_test)}")
    return model

In [ ]:
X = df[["sqfeet"]]
y = df[["price"]]
doit(X, y, "all the rows sqfeet to price")

This model is horribly bad.

### A few ideas how to improve it:
* Remove the outliers in the size of the housing
* Remove the outliers in the price
* Cluster the data (there might be a much better prediction when the size of the housings are similar
* Take into account other features. (how to take into account a feature with categorial values. e.g. state and type and with discrete values e.g. number of rooms)

## Type
* What kind of values does this have?
* How many of each value

In [ ]:
df['type']

In [ ]:
df.head(100).groupby('type').nunique()

In [ ]:
df[['id', 'type']].head(100).groupby('type').nunique()

In [ ]:
df[['id', 'type']].groupby('type').nunique()

We could try the same model for specific type of housing

In [ ]:
for housing_type in ['apartment', 'condo', 'house', 'duplex', 'townhouse']:
    subset = df[df['type'] == housing_type]
    X = subset[["sqfeet"]]
    y = subset[["price"]]
    doit(X, y, f"{housing_type} only: sqfeet to price")
    print("-----")

## State

In [ ]:
df[['id', 'state']].groupby('state').nunique()

## Region

In [ ]:
x = df[['id', 'region']].groupby('region').nunique()
type(x)
x = x.rename(columns={"id": "count"})
x.sort_values(by=['count'], ascending=False)

In [ ]:
df[df['region'] == 'abilene'].count()

## Prices
* Tryting to figure out the range of the prices
* Remove rows where the price does not make sense
* Remove outliers

In [ ]:
print(f"max: {df['price'].max()}")
print(f"min: {df['price'].min()}")
df["price"].describe()

In [ ]:
df.plot.scatter(x='sqfeet', y='price', s=5);

In [ ]:
print(f"Number of rows: {df['price'].count()}")
print(f"Number of price==0 {df[df['price'] == 0]['price'].count()}")
print(f"Number of price!=0 {df[df['price'] != 0]['price'].count()}")
print(f"Number of price<500 {df[df['price'] < 500]['price'].count()}")
print(f"Number of price<2000 {df[df['price'] < 2000]['price'].count()}")
print(f"Number of price>100000 {df[df['price'] > 100000]['price'].count()}")

In [ ]:
# The coordinates of the Empire State building:
latitude = 40.748817
longitude = -73.985428
a = abs(df['lat']-latitude)<0.2
b = abs(df['long']-longitude)<0.2
near_by = df[a & b]
print(near_by.count())
X = near_by[["sqfeet"]]
y = near_by[["price"]]
doit(X, y, f"near by the Empire State building only: sqfeet to price")

# Tried to check the URL of a listing, but it is not available (any more?)
# near_by.iloc[0]['url']

#print(f"Number of rows near-by: {df[(abs(df['lat']-latitude)<0.5) & (abs(df['long']-longitude)<0.5)].count()}")

In [ ]:
sane_price = df[ (500 < df['price']) & (df['price'] < 2000) ]
sane_price.count()

In [ ]:
sane_price.plot.scatter(x='sqfeet', y='price', s=5);

In [ ]:
print(f"Number of rows: {df['sqfeet'].count()}")
print(f"Number of sqfeet==NaN {df[df['sqfeet'].isnull()]['sqfeet'].count()}")
print(f"Number of sqfeet==0 {df[df['sqfeet'] == 0]['sqfeet'].count()}")
print(f"Number of sqfeet<400 {df[df['sqfeet'] < 400]['sqfeet'].count()}")
print(f"Number of sqfeet>2000 {df[df['sqfeet'] > 2000]['sqfeet'].count()}")
print(f"Number of sqfeet!=0 {df[df['sqfeet'] != 0]['sqfeet'].count()}")

In [ ]:
sane_sqfeet = df[ (400 < df['sqfeet']) & (df['sqfeet'] < 2000) ]
sane_sqfeet.count()

In [ ]:
sane_sqfeet.plot.scatter(x='sqfeet', y='price', s=5);

In [ ]:
sane = sane_sqfeet[ (500 < sane_sqfeet['price']) & (sane_sqfeet['price'] < 3000) ]
sane.count()

In [ ]:
#sane = sane[ sane['state'] == 'ca' ]
sane.plot.scatter(x='sqfeet', y='price', s=5);

In [ ]:
X = sane[["sqfeet"]]
y = sane[["price"]]
doit(X, y, f"sane: sqfeet to price")

In [ ]:
sane_in_region = sane[sane['region'] == 'jacksonville']
#sane_in_region = sane[sane['region'] == 'columbus']
#sane_in_region = sane[sane['region'].isin(['jacksonville', 'columbus']) ]
sane_in_region.count()
sane_in_region.describe()
sane_in_region.plot.scatter(x='sqfeet', y='price', s=5);

In [ ]:
X = sane_in_region[["sqfeet"]]
y = sane_in_region[["price"]]
doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
X = sane_in_region[["sqfeet", "beds"]]
y = sane_in_region[["price"]]
doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
X = sane_in_region[["sqfeet", "beds", "baths"]]
y = sane_in_region[["price"]]
mymodel = doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
sane_in_region[["price", "sqfeet", "beds", "baths"]].describe()

In [ ]:
sane_in_region[["price", "sqfeet", "beds", "baths"]].head(10)

In [ ]:
mymodel.predict([[1000, 1, 1], [1000, 2, 2], [1000, 1, 2], [760, 1, 1]])
mymodel.predict(sane_in_region[["sqfeet", "beds", "baths"]].head(10)) - sane_in_region.head(10)["price"]


In [ ]:
X = sane_in_region[["sqfeet", "beds", "baths", "cats_allowed"]] #
y = sane_in_region[["price"]]
doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
X = sane_in_region[["sqfeet", "beds", "baths", "cats_allowed", "dogs_allowed"]] # smoking_allowed 	wheelchair_access
y = sane_in_region[["price"]]
doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
X = sane_in_region[["sqfeet", "beds", "baths", "cats_allowed", "dogs_allowed", "smoking_allowed"]] #  	wheelchair_access
y = sane_in_region[["price"]]
doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
X = sane_in_region[["sqfeet", "beds", "baths", "cats_allowed", "dogs_allowed", "smoking_allowed", "wheelchair_access"]]
y = sane_in_region[["price"]]
doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
sane_in_region.shape

In [ ]:
sane_in_region_clean = sane_in_region.dropna(axis=0, subset=['lat', 'long'])

In [ ]:
sane_in_region_clean.shape

In [ ]:
X = sane_in_region_clean[["sqfeet", "beds", "baths", "cats_allowed", "dogs_allowed", "smoking_allowed", "wheelchair_access", "lat", "long"]]
y = sane_in_region_clean[["price"]]
doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
fields = ["sqfeet", "beds", "baths", "cats_allowed", "dogs_allowed", "smoking_allowed", "wheelchair_access", "lat", "long"]
df_clean = df.dropna(axis=0, subset=fields)
X = df_clean[fields]
y = df_clean[["price"]]
doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
fields = ["sqfeet", "beds", "baths", "cats_allowed", "dogs_allowed", "smoking_allowed", "wheelchair_access", "lat", "long"]
sane_clean = sane.dropna(axis=0, subset=fields)
X = sane_clean[fields]
y = sane_clean[["price"]]
x_model = doit(X, y, f"sane_in_region: sqfeet to price")

In [ ]:
x_model.predict(sane_clean[fields].head(20)) - sane_clean.head(20)["price"]